# Losses, NAB scores and classification metrics analysis

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve, roc_auc_score

In [2]:
scores = pd.read_csv("results/scores.csv")
losses_log_total = pd.read_csv("results/losses_log_total.csv")
losses_square_total = pd.read_csv("results/losses_square_total.csv")

In [3]:
experts = [
    "knncad",
    "numentaTM",
    "twitterADVec",
    "skyline",
    "earthgeckoSkyline",
    "numenta",
    "bayesChangePt",
    "null",
    "expose",
    "relativeEntropy",
    "htmjava",
    "randomCutForest",
    "random",
    "contextOSE",
    "windowedGaussian",
]

## Total logarithmic and square losses

In [4]:
losses_log = losses_log_total.filter(regex="^loss", axis=1).sum()
losses_square = losses_square_total.filter(regex="^loss", axis=1).sum()
losses_log.index = [i.replace("loss_", "") for i in losses_log.index]
losses_square.index = [i.replace("loss_", "") for i in losses_square.index]

## Classification metrics

In [5]:
auc_score = pd.DataFrame(
    index=[
        "realAWSCloudwatch",
        "realTraffic",
        "realAdExchange",
        "realTweets",
        "artificialWithAnomaly",
        "realKnownCause",
        "Total",
    ]
)
alg_list = scores.filter(regex="^score", axis=1).columns.tolist()
alg_list = [i.replace("score_", "") for i in alg_list]
f1_score = auc_score.copy()
class_score = auc_score.copy()
thresholds_mat = auc_score.copy()
EPSILON = 1e-7
for folder_name in auc_score.index:
    for alg_ind in alg_list:
        if folder_name == "Total":
            auc_score.loc["Total", alg_ind] = roc_auc_score(
                scores["label"], scores[f"score_{alg_ind}"]
            )
            precision, recall, thresholds = precision_recall_curve(
                scores["label"], scores[f"score_{alg_ind}"]
            )
            precision[(precision == 0) & (recall == 0)] = EPSILON
            f1 = 2 * precision * recall / (precision + recall)
            f1_score.loc["Total", alg_ind] = np.max(f1)
            thresholds_mat.loc["Total", alg_ind] = (
                thresholds[f1[1:] == np.max(f1[1:])]
            ).item()
            preds = np.where(
                scores[f"score_{alg_ind}"]
                >= (thresholds[f1[1:] == np.max(f1[1:])]).item(),
                1,
                0,
            )
            class_score.loc["Total", alg_ind] = (
                sum(preds == scores["label"]) / scores.shape[0]
            )
        else:
            scores_folder = scores[scores["folder_name"] == folder_name]
            auc_score.loc[folder_name, alg_ind] = roc_auc_score(
                scores_folder["label"], scores_folder[f"score_{alg_ind}"]
            )
            precision, recall, thresholds = precision_recall_curve(
                scores_folder["label"], scores_folder[f"score_{alg_ind}"]
            )
            precision[(precision == 0) & (recall == 0)] = EPSILON
            f1 = 2 * precision * recall / (precision + recall)
            f1_score.loc[folder_name, alg_ind] = np.max(f1)
            thresholds_mat.loc[folder_name, alg_ind] = (
                thresholds[f1[1:] == np.max(f1[1:])]
            ).item()
            preds = np.where(
                scores_folder[f"score_{alg_ind}"]
                >= (thresholds[f1[1:] == np.max(f1[1:])]).item(),
                1,
                0,
            )
            class_score.loc[folder_name, alg_ind] = (
                sum(preds == scores_folder["label"]) / scores_folder.shape[0]
            )

In [6]:
metrics_total = pd.concat(
    [
        auc_score.T["Total"],
        f1_score.T["Total"],
        class_score.T["Total"],
        losses_log,
        losses_square,
    ],
    axis=1,
)
metrics_total.columns = [
    "auc",
    "f1_score",
    "class_accuracy",
    "log_loss",
    "square_loss",
]

In [7]:
share_names = auc_score.filter(
    regex="^Fixed|^Variable", axis=1
).columns.tolist()
metrics_delays = metrics_total.loc[share_names]
metrics_delays["alpha_delay"] = metrics_delays.index.to_series().replace(
    r"^(Fixed|Variable)", "", regex=True
)
metrics_delays[["alpha", "delay"]] = metrics_delays["alpha_delay"].str.split(
    "d", expand=True
)
metrics_delays["type_alpha"] = (
    metrics_delays.index.str.split(r"\d+", n=1, expand=True).get_level_values(
        0
    )
    + metrics_delays["alpha"]
)

In [8]:
pivots = {
    key: pd.pivot_table(
        metrics_delays.sort_values("alpha"),
        values=key,
        index="type_alpha",
        columns="delay",
    )
    for key in ("auc", "f1_score", "class_accuracy", "log_loss", "square_loss")
}

In [9]:
def my_sorter(index):
    fixed = index.str.replace(r"\d+", "", regex=True)
    alpha = index.str.replace(r"\D+", "", regex=True).astype(int)
    return pd.MultiIndex.from_arrays((fixed, alpha))

## Area under curve

In [10]:
pivots["auc"].sort_index(key=my_sorter).round(3)

delay,0,1,2,3,4
type_alpha,,,,,
Fixed0,0.997,0.938,0.809,0.696,0.771
Fixed1,0.998,0.970,0.922,0.833,0.894
Fixed5,0.998,0.974,0.936,0.869,0.912
Fixed10,0.998,0.975,0.940,0.881,0.918
Fixed30,0.998,0.975,0.942,0.893,0.920
Variable0,0.991,0.826,0.657,0.581,0.639
Variable1,0.997,0.966,0.926,0.874,0.904
Variable5,0.998,0.976,0.949,0.906,0.925
Variable10,0.998,0.978,0.954,0.909,0.929


In [11]:
metrics_total.loc[experts]["auc"].sort_values(ascending=False).head(3).round(3)

randomCutForest    0.616
knncad             0.600
skyline            0.566
Name: auc, dtype: float64

## F-score

In [12]:
pivots["f1_score"].sort_index(key=my_sorter).round(3)

delay,0,1,2,3,4
type_alpha,,,,,
Fixed0,0.978,0.684,0.418,0.296,0.372
Fixed1,0.988,0.832,0.651,0.455,0.596
Fixed5,0.990,0.863,0.714,0.540,0.667
Fixed10,0.990,0.877,0.744,0.584,0.701
Fixed30,0.984,0.897,0.788,0.650,0.748
Variable0,0.961,0.521,0.319,0.237,0.300
Variable1,0.979,0.809,0.677,0.553,0.642
Variable5,0.983,0.870,0.775,0.672,0.740
Variable10,0.984,0.888,0.802,0.690,0.763


In [13]:
metrics_total.loc[experts]["f1_score"].sort_values(ascending=False).head(
    3
).round(3)

randomCutForest    0.233
htmjava            0.218
knncad             0.216
Name: f1_score, dtype: float64

## Classification accuracy

In [14]:
pivots["class_accuracy"].sort_index(key=my_sorter).round(3)

delay,0,1,2,3,4
type_alpha,,,,,
Fixed0,0.996,0.939,0.886,0.824,0.878
Fixed1,0.998,0.968,0.934,0.875,0.923
Fixed5,0.998,0.975,0.943,0.901,0.933
Fixed10,0.998,0.977,0.950,0.914,0.941
Fixed30,0.997,0.981,0.960,0.932,0.952
Variable0,0.993,0.909,0.873,0.855,0.869
Variable1,0.996,0.965,0.938,0.918,0.933
Variable5,0.997,0.976,0.957,0.936,0.951
Variable10,0.997,0.979,0.962,0.939,0.955


In [15]:
metrics_total.loc[experts]["class_accuracy"].sort_values(ascending=False).head(
    3
).round(3)

numenta             0.866
numentaTM           0.864
windowedGaussian    0.862
Name: class_accuracy, dtype: float64

## Logarithmic loss

In [16]:
(pivots["log_loss"].sort_index(key=my_sorter) / 10 ** 3).round(1)

delay,0,1,2,3,4
type_alpha,,,,,
Fixed0,12.1,61.1,95.3,106.2,101.4
Fixed1,11.8,36.3,61.5,87.5,69.9
Fixed5,19.9,37.6,56.6,78.0,63.2
Fixed10,28.4,42.6,58.3,76.4,63.8
Fixed30,56.8,64.2,73.7,84.6,77.0
Variable0,17.4,90.0,109.6,115.4,112.3
Variable1,12.4,40.6,59.2,75.6,64.8
Variable5,11.6,34.1,49.7,66.0,56.1
Variable10,11.3,32.9,48.8,66.2,55.9


In [17]:
(
    metrics_total.loc[experts]["log_loss"].sort_values(ascending=True).head(3)
    / 10 ** 3
).round(1)

randomCutForest    107.5
htmjava            153.9
numentaTM          161.4
Name: log_loss, dtype: float64

## Square loss

In [18]:
(pivots["square_loss"].sort_index(key=my_sorter) / 10 ** 3).round(1)

delay,0,1,2,3,4
type_alpha,,,,,
Fixed0,2.9,16.5,26.4,29.0,27.6
Fixed1,2.5,9.7,17.4,24.9,19.6
Fixed5,3.0,8.9,15.1,21.8,17.1
Fixed10,4.1,9.2,14.6,20.7,16.4
Fixed30,10.3,13.4,17.1,21.2,18.3
Variable0,3.8,23.5,28.9,30.0,29.5
Variable1,2.9,10.7,16.4,21.3,17.9
Variable5,2.8,8.8,13.6,18.6,15.2
Variable10,2.7,8.6,13.6,18.8,15.3


In [19]:
(
    metrics_total.loc[experts]["square_loss"]
    .sort_values(ascending=True)
    .head(3)
    / 10 ** 3
).round(1)

randomCutForest    29.4
htmjava            31.5
numentaTM          31.8
Name: square_loss, dtype: float64